<a href="https://colab.research.google.com/github/harjeet88/LLM_experiemnts/blob/main/RAG_v2/LLM_flan_hp_faster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sqlite3
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline

In [2]:
# 1. Setup SQLite Database
def setup_db(db_name="rag_hp_db.sqlite"):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS documents (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            content TEXT,
            embedding BLOB
        )
    """)
    conn.commit()
    return conn

In [10]:
# 2. Embed Documents and Store in DB
def embed_and_store(conn, documents, embedding_model):
    cursor = conn.cursor()
    for doc in documents:
        #embedding = embedding_model.encode(doc, convert_to_tensor=True).numpy().tobytes()
        embedding = embedding_model.encode(doc, convert_to_tensor=True).cpu().numpy().tobytes() # Added .cpu() before .numpy()
        cursor.execute("INSERT INTO documents (content, embedding) VALUES (?, ?)", (doc, embedding))
    conn.commit()


In [11]:
# 3. Retrieve Relevant Documents
def retrieve_documents(conn, query, embedding_model, top_k=3):
    query_embedding = embedding_model.encode(query, convert_to_tensor=True)
    cursor = conn.cursor()
    cursor.execute("SELECT id, content, embedding FROM documents")
    rows = cursor.fetchall()
    similarities = []
    for row in rows:
        doc_embedding = embedding_model.encode(row[1], convert_to_tensor=True) #using content to create embedding.
        similarity = util.pytorch_cos_sim(query_embedding, doc_embedding).item()
        similarities.append((similarity, row[1])) #store similarity and content.
    similarities.sort(key=lambda x: x[0], reverse=True)
    return [doc for _, doc in similarities[:top_k]]

In [12]:
# 4. Generate Answer with LLM
def generate_answer(query, retrieved_documents, llm_model):
    context = "\n".join(retrieved_documents)
    prompt = f"Context:\n{context}\n\nQuestion: {query}\n\nAnswer:"
    answer = llm_model(prompt)[0]['generated_text']
    return answer

In [13]:
# Main Execution
def rag_pipeline(query, documents, embedding_model_name="all-mpnet-base-v2", llm_model_name="google/flan-t5-large"):
    conn = setup_db()
    embedding_model = SentenceTransformer(embedding_model_name)
    embed_and_store(conn, documents, embedding_model)
    retrieved_docs = retrieve_documents(conn, query, embedding_model)
    llm_model = pipeline("text2text-generation", model=llm_model_name)
    answer = generate_answer(query, retrieved_docs, llm_model)
    conn.close()
    return answer

In [14]:
# Example Usage
documents = [
    "The capital of France is Paris.",
    "The Eiffel Tower is in Paris.",
    "London is the capital of England.",
    "Dogs are loyal animals.",
    "Cats are independent animals."
]

In [15]:
# HP Usage
documents = [
    "Harry Potter is a series of seven fantasy novels written by British author J. K. Rowling. The novels chronicle the lives of a young wizard, Harry Potter, and his friends, Hermione Granger and Ron Weasley, all of whom are students at Hogwarts School of Witchcraft and Wizardry. The main story arc concerns Harry's conflict with Lord Voldemort, a dark wizard who intends to become immortal, overthrow the wizard governing body known as the Ministry of Magic, and subjugate all wizards and Muggles (non-magical people).",
    "Harry learns that his parents, Lily and James Potter, also had magical powers and were murdered by the dark wizard Lord Voldemort when Harry was a baby.",
    "He gains the friendship of Ron Weasley, a member of a large but poor wizarding family, and Hermione Granger, a witch of non-magical, or Muggle, parentage.",
    "he first book concludes with Harry's confrontation with Voldemort, who, in his quest to regain a body, yearns to possess the Philosopher's Stone, a substance that bestows everlasting life.",
    "Tom riddle is actual Voldemort. Harry learns from a drunken Slughorn that he used to teach Tom Riddle, and that Voldemort divided his soul into pieces, creating a series of Horcruxes."
]

In [16]:
query = "Where is the Eiffel Tower?"
answer = rag_pipeline(query, documents)
print(answer)

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cuda:0


unanswerable


In [17]:
query2 = "what are dogs known for?"
answer2 = rag_pipeline(query2, documents)
print(answer2)

Device set to use cuda:0


unanswerable


In [18]:
query2 = "who are friends of harry potter?"
answer2 = rag_pipeline(query2, documents)
print(answer2)

Device set to use cuda:0


Ron Weasley


In [21]:
query4 = "who killed harry potter's parents?"
answer4 = rag_pipeline(query4, documents)
print(answer4)

Device set to use cuda:0


Lord Voldemort


In [24]:
query5 = "who are two friends of harry potter apart from Ron Weasley?"
answer5 = rag_pipeline(query5, documents)
print(answer5)

Device set to use cuda:0


Hermione Granger
